
# 🧠 Lab 19: Prompt Engineering vs Context Engineering (Hallucination Demo)

**Objective:**  
Understand how *Prompt Engineering* (improving instructions) and *Context Engineering* (providing real data) affect an LLM’s ability to give *factual vs. hallucinated* responses.

This lab compares:
- Zero-shot prompting
- Few-shot prompting
- Chain-of-Thought prompting
- Retrieval-Augmented Generation (RAG)

The question chosen is **knowledge-based**, so we can easily see where the model *hallucinates* without context.


In [1]:

# === Step 1: Setup NVIDIA API Key ===
import os, json, requests

# Set your NVIDIA API key (replace with your actual key)
os.environ['NVIDIA_API_KEY'] = "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"

NVIDIA_ENDPOINT = "https://integrate.api.nvidia.com/v1/chat/completions"
MODEL = "meta/llama3-8b-instruct"

def ask_llm(prompt, max_tokens=200):
    headers = {
        "Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": 0.7
    }
    response = requests.post(NVIDIA_ENDPOINT, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error {response.status_code}: {response.text}"



---
## 🧩 Step 2: The Test Question

We’ll ask the model a factual question **it’s unlikely to know correctly without context**:

> *"What is the battery capacity of the HPE ProLiant DL380a Gen12 server?"*

This detail exists in documentation, but the model was *not trained* with proprietary product specs — perfect for testing hallucination.


In [2]:

# === Step 3: Zero-Shot Prompting ===
prompt_zero = "What is the battery capacity of the HPE ProLiant DL380a Gen12 server?"
print("🧠 Zero-shot Prompt:")
print(prompt_zero)
print("\n---\nResponse:\n")
print(ask_llm(prompt_zero))


🧠 Zero-shot Prompt:
What is the battery capacity of the HPE ProLiant DL380a Gen12 server?

---
Response:

The HPE ProLiant DL380a Gen12 server can come with various battery configurations, and the battery capacity depends on the specific model and configuration. Here are some examples:

* The HPE ProLiant DL380a Gen12 server with a 2U rackmount design can support up to 16 hot-pluggable batteries, each with a capacity of 2.4Ah. This results in a total battery capacity of 38.4Ah.
* The HPE ProLiant DL380a Gen12 server with a 4U rackmount design can support up to 32 hot-pluggable batteries, each with a capacity of 2.4Ah. This results in a total battery capacity of 76.8Ah.

It's worth noting that the battery capacity is dependent on the specific configuration and model of the server, so it's best to check the documentation or contact HPE support for specific information on the battery capacity of your server.


In [3]:

# === Step 4: Few-Shot Prompting ===
prompt_few = '''You are an expert in enterprise servers. Here are examples:

Q: What is the maximum memory capacity of HPE DL380 Gen11?
A: It supports up to 8TB DDR5 memory.

Q: What is the GPU support for HPE DL325 Gen12?
A: It supports up to 4 double-wide GPUs.

Now, answer:
Q: What is the battery capacity of the HPE ProLiant DL380a Gen12 server?
'''
print("💡 Few-shot Prompt:")
print(prompt_few)
print("\n---\nResponse:\n")
print(ask_llm(prompt_few))


💡 Few-shot Prompt:
You are an expert in enterprise servers. Here are examples:

Q: What is the maximum memory capacity of HPE DL380 Gen11?
A: It supports up to 8TB DDR5 memory.

Q: What is the GPU support for HPE DL325 Gen12?
A: It supports up to 4 double-wide GPUs.

Now, answer:
Q: What is the battery capacity of the HPE ProLiant DL380a Gen12 server?


---
Response:

The HPE ProLiant DL380a Gen12 server has a 530W 92% efficient hot-pluggable redundant power supply with a battery capacity of 700W.


In [4]:

# === Step 5: Chain-of-Thought Prompting ===
prompt_cot = '''Let's reason step by step.

1. The HPE ProLiant DL380a Gen12 is a compute server.
2. Servers do not use a battery like laptops do.
3. However, they may have a Smart Storage Battery for RAID controllers.

Based on that, answer: What is the battery capacity of the HPE ProLiant DL380a Gen12 server?
'''
print("🧩 Chain-of-Thought Prompt:")
print(prompt_cot)
print("\n---\nResponse:\n")
print(ask_llm(prompt_cot))


🧩 Chain-of-Thought Prompt:
Let's reason step by step.

1. The HPE ProLiant DL380a Gen12 is a compute server.
2. Servers do not use a battery like laptops do.
3. However, they may have a Smart Storage Battery for RAID controllers.

Based on that, answer: What is the battery capacity of the HPE ProLiant DL380a Gen12 server?


---
Response:

Let's reason step by step as you suggested!

From statement 1, we know that the HPE ProLiant DL380a Gen12 is a compute server.

From statement 2, we know that servers do not use a battery like laptops do.

However, from statement 3, we know that servers may have a Smart Storage Battery for RAID controllers.

But, we are asked about the battery capacity of the HPE ProLiant DL380a Gen12 server, not if it has a battery or not. Since we don't have any information about the specific battery capacity of this server, we cannot provide an answer.

Therefore, the answer is: Unknown or Not Provided.


In [5]:

# === Step 6: Context Engineering (Simulated RAG) ===
# Simulated context retrieved from product documentation
context = '''According to the official HPE ProLiant DL380a Gen12 QuickSpecs:
The server includes a Smart Storage Battery (96W) that supports HPE Smart Array Controllers.
This battery maintains data integrity during unexpected power loss.'''

prompt_context = f"Use the context below to answer accurately.\nContext:\n{context}\n\nQuestion: What is the battery capacity of the HPE ProLiant DL380a Gen12 server?"
print("📚 Context-Enhanced Prompt (RAG Simulation):")
print(prompt_context)
print("\n---\nResponse:\n")
print(ask_llm(prompt_context))


📚 Context-Enhanced Prompt (RAG Simulation):
Use the context below to answer accurately.
Context:
According to the official HPE ProLiant DL380a Gen12 QuickSpecs:
The server includes a Smart Storage Battery (96W) that supports HPE Smart Array Controllers.
This battery maintains data integrity during unexpected power loss.

Question: What is the battery capacity of the HPE ProLiant DL380a Gen12 server?

---
Response:

According to the context, the battery capacity of the HPE ProLiant DL380a Gen12 server is 96W.
